# Quantum Fourier Transform with BQSKit
---
![bqskit](https://bqskit.lbl.gov/wp-content/uploads/sites/6/2021/02/BQSKit_header-1.png)

The Berkeley Quantum Synthesis Toolkit (BQSKit) is a powerful and portable quantum compiler framework. This tutorial explores how to use BQSKit to compile the quantum phase estimation algorithm. 

To **install** BQSKit and other required packages, follow the instructions in the [README](https://github.com/BQSKit/bqskit-tutorial/blob/main/README.md).

## Fourier transform for wavefunction amplitude <a id='introduction'></a>


The Fourier transform occurs in many different areas ranging from signal processing, data compression, complexity theory, to high energy physics. It is an integral transform that converts a function into a form that describes the frequencies present in the original function. The output of the transform is a complex-valued function of frequency. The Fourier transform is analogous to decomposing the sound of a musical chord into the intensities of its constituent pitches. The mathematical expression for the transform is,

$$
\widehat{f}(\xi) = \int_{-\infty}^{\infty} f(x)\ e^{-i 2\pi \xi x}\,dx.
$$

The inverse Fourier transform is,

$$f(x) = \int_{-\infty}^{\infty} \widehat f(\xi)\ e^{i 2 \pi \xi x}\,d\xi.$$


The discrete Fourier transform acts on a vector $(x_0, ..., x_{N-1})$ and maps it to the vector $(y_0, ..., y_{N-1})$ according to the formula


$$y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_j\omega_N^{jk}$$


where $\omega_N^{jk} = e^{2\pi i \frac{jk}{N}}$. The quantum Fourier transform (QFT) is the quantum implementation of the discrete Fourier transform over the amplitudes of a wavefunction $|X\rangle = \sum_k x_k | k \rangle$, to the wavefunction $|Y\rangle = \sum_k y_k | k \rangle$, based on the same formula.
This can also be stated as unitary transformation, 
$U_{QFT}=\frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}\sum_{k=0}^{N-1}\omega^{jk}_{N}\vert k \rangle \langle j\vert$.

Thus,

$$\begin{equation}
\begin{split}
U_{QFT} |X\rangle =& \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}\sum_{k=0}^{N-1}\omega^{jk}_{N}|k \rangle \langle j| \\
=& \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} 2^{2\pi i (\sum_{k=1}^n x_k/2^n)} |x_1...x_n \rangle \\
=&\frac{1}{2^n}\bigotimes_{k=1}^n \left(| 0 \rangle + e^{2\pi i \frac{x}{2^k}} | 1\rangle \right)\\
=& \frac{1}{\sqrt{N}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right) 
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right) 
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right) 
\end{split}
\end{equation}
$$

### Single  qubit QFT <a id='example1'></a>

Consider how the QFT operator as defined above acts on a single qubit state $\vert\psi\rangle = \alpha \vert 0 \rangle + \beta \vert 1 \rangle$. In this case, $x_0 = \alpha$, $x_1 = \beta$, and $n = 1$. Then,



$$y_0 = \frac{1}{\sqrt{2}}\left(    \alpha \exp\left(2\pi i\frac{0\times0}{2}\right) + \beta \exp\left(2\pi i\frac{1\times0}{2}\right)      \right) = \frac{1}{\sqrt{2}}\left(\alpha + \beta\right),$$



and



$$y_1 = \frac{1}{\sqrt{2}}\left(    \alpha \exp\left(2\pi i\frac{0\times1}{2}\right) + \beta \exp\left(2\pi i\frac{1\times1}{2}\right)      \right) = \frac{1}{\sqrt{2}}\left(\alpha - \beta\right).$$



The final resulting state is 



$$
\begin{equation}
\begin{split}
U_{QFT}\vert\psi\rangle =& \frac{1}{\sqrt{2}}(\alpha + \beta) \vert 0 \rangle + \frac{1}{\sqrt{2}}(\alpha - \beta)  \vert 1 \rangle\\
=& \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \vert\psi\rangle \\
=& H \vert\psi\rangle
\end{split}
\end{equation}
$$



This operation is exactly the result of applying the Hadamard operator $H$ on the qubit.

##  Multiple qubit circuit implementation <a name="circuit"></a>

Two are the main gates used for the circuit. The first one is a single-qubit Hadamard gate $H$, and the second is a twoo-qubit controlled rotation $CROT_k$ given in block-diagonal form as 

$$CR_k = \left[\begin{matrix}
I&0\\
0&U_k\\
\end{matrix}\right]$$

where 

$$U_k = \left[\begin{matrix}
1&0\\
0&\exp\left(\frac{2\pi i}{2^k}\right)\\
\end{matrix}\right]$$

The action of $CR_k$ on a two-qubit state is given by



$$CR_k\vert 0x_j\rangle = \vert 0x_j\rangle, \ CR_k\vert 1x_j\rangle = \exp\left( \frac{2\pi i}{2^k}x_j \right)\vert 1x_j\rangle$$



We start with an n-qubit input state $\vert x_1x_2\ldots x_n\rangle$.

<ol>
<li> After the first Hadamard gate on qubit 1, the state is transformed from the input state to 

$$
H_1\vert x_1x_2\ldots x_n\rangle = 
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + \exp\left(\frac{2\pi i}{2}x_1\right)\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> After the $U_2$ gate on qubit 1 controlled by qubit 2, the state is transformed to

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + \exp\left(\frac{2\pi i}{2^2}x_2 + \frac{2\pi i}{2}x_1\right)\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> Repeating the process up to qubit $n$, the state becomes

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x_n + 
\frac{2\pi i}{2^{n-1}}x_{n-1} + 
\ldots + 
\frac{2\pi i}{2^2}x_2 + 
\frac{2\pi i}{2}x_1
\right)
\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

Noting that 

$$
x = 2^{n-1}x_1 + 2^{n-2}x_2 + \ldots + 2^1x_{n-1} + 2^0x_n
$$

we can write the above state as 

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x 
\right)
\vert1\rangle\right]
\otimes
\vert x_2x_3\ldots x_n\rangle
$$

<li> After the application of a similar sequence of gates for qubits $2\ldots n$, we find the final state to be:

$$
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^n}x 
\right)
\vert1\rangle\right]
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{n-1}}x 
\right)
\vert1\rangle\right]
\otimes
\ldots
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{2}}x 
\right)
\vert1\rangle\right]
\otimes
\frac{1}{\sqrt{2}}
\left[\vert0\rangle + 
\exp\left(
\frac{2\pi i}{2^{1}}x 
\right)
\vert1\rangle\right]
$$

This is $|Y\rangle$ the order of the qubits is reversed in the output state. To obtain the right order, we perform swap gates. 
    
    
TODO: include comments about numeric precision!    
    
The QFT function is written as follows, 
</ol>

In [26]:
from bqskit import Circuit
from bqskit.ir.gates import XGate, HGate, CPGate, SwapGate
import numpy as np

# build the QFT function 
def QFT(qcirc,n,initial=0,inverse=False,swap=True,nmax=16): # add option for final swaps, check that precision option as it should
    """
        Args:
            
            qcirc: quantum circuit instance in BQSkit
            n: the number of consecutive qubits QFT will act on
            initial: the index of the first qubit QFT will act on
            inverse: whether to apply QFT or QFT^(-1)
            swap: whether to apply swap gates to revert the qubit order back to the original, 
                  by default the order is reversered
            nmax: the maximal value of n allowed for computations as precision is limited
        Raises:
            Warning: if n>nmax
            
        Returns: 
            No return, all gates required for QFT are applied on qicrc and the circuit is modified accordingly  
    """
    assert n <= qcirc.num_qudits
    if n>nmax:
        print(" n = "+str(n)+" is higher than "+str(nmax)+", no QFT will be performed!")
    
    if not inverse:
        # repeated H and CR_k applications
        for i in range(n-1+initial,-1+initial,-1):
            qcirc.append_gate(HGate(),i)
            p=0
            for j in range(initial,i):
                qcirc.append_gate(CPGate(),[i-j-1,i],[np.pi/(2**p)]) # apply control phases with the power of 2^distance
                p+=1
            
        if swap:
            # swap gates to fix the order of the qubits
            for i in range(initial,initial+int(n//2)):
                qcirc.append_gate(SwapGate(),[i,n-i-1+initial]) # swap the first with last etc until the middle of the circuit
    else: # the inverse circuit is also reveresed in order of gate applications    
        if swap:
            # swap gates to fix the order of the qubits
            for i in range(initial,initial+int(n//2)):
                qcirc.append_gate(SwapGate(),[i,n-i-1+initial]) # swap the first with last etc until the middle of the circuit  
            
        # repeated H and CR_k applications with revesed order and sign for the phases
        for i in range(initial,n+initial):
            p=i
            for j in range(initial,i):
                p-=1
                qcirc.append_gate(CPGate(),[j,i],[-np.pi/(2**p)]) # apply control phases with the power of 2^distance
            qcirc.append_gate(HGate(),i)

## Example: 4 qubit QFT

We start with $\vert \Psi \rangle = \vert 0101 \rangle$.

In [33]:
# initialize the circuit
qcirc=Circuit(4)

# prepare the initial state
qcirc.append_gate(XGate(),1)
qcirc.append_gate(XGate(),3)

# perform qft
QFT(qcirc,4,inverse=False,swap=False,nmax=16)
qcirc

Circuit(4)
	[None, XGate@(1,), None, XGate@(3,)]
	[None, None, None, HGate@(3,)]
	[None, None, CPGate([3.141592653589793])@(2, 3), CPGate([3.141592653589793])@(2, 3)]
	[None, CPGate([1.5707963267948966])@(1, 3), HGate@(2,), CPGate([1.5707963267948966])@(1, 3)]
	[CPGate([0.7853981633974483])@(0, 3), CPGate([3.141592653589793])@(1, 2), CPGate([3.141592653589793])@(1, 2), CPGate([0.7853981633974483])@(0, 3)]
	[CPGate([1.5707963267948966])@(0, 2), HGate@(1,), CPGate([1.5707963267948966])@(0, 2), None]
	[CPGate([3.141592653589793])@(0, 1), CPGate([3.141592653589793])@(0, 1), None, None]
	[HGate@(0,), None, None, None]

In [38]:
from bqskit import compile

# Building Rigetti's native gate set
from bqskit.ir.gates import U1qGate, RZGate, ZZGate
gate_set = {RZGate(), ZZGate(), U1qGate()} 

# Build a MachineModel with this gate set
# and the same number of qubits as the circuit
from bqskit import MachineModel
model = MachineModel(qcirc.num_qudits, gate_set=gate_set)

# # Compile the circuit with optimization level 3
out_circuit = compile(qcirc, model=model, optimization_level=3)
print("Gate Counts:", out_circuit.gate_counts)

# Print new statistics
print("Compiled Circuit Statistics")
print("Gate Counts:", out_circuit.gate_counts)
print("Logical Connectivity:", out_circuit.coupling_graph)

/opt/homebrew/anaconda3/envs/bqskit/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/bqskit/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/bqskit/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/bqskit/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/bqskit/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/anaconda3/envs/bqskit/lib/py

Gate Counts: {U1qGate: 14, ZZGate: 9, RZGate: 6}
Compiled Circuit Statistics
Gate Counts: {U1qGate: 14, ZZGate: 9, RZGate: 6}
Logical Connectivity: CouplingGraph({(0, 1), (1, 2), (0, 3), (2, 3), (0, 2), (1, 3)})


In [39]:
out_circuit

Circuit(4)
	[None, None, U1qGate([1.570796344593084, 1.5707963064008534])@(2,), U1qGate([7.853981626304372, 20.420352238330118])@(3,)]
	[None, ZZGate@(1, 2), ZZGate@(1, 2), None]
	[None, None, U1qGate([4.7123889789500755, 3.1415926515049093])@(2,), None]
	[None, None, ZZGate@(2, 3), ZZGate@(2, 3)]
	[None, None, RZGate([-1.8583570386147872])@(2,), U1qGate([1.5707963225517572, 33.629362028085346])@(3,)]
	[None, ZZGate@(1, 3), U1qGate([2.278219306461859, 4.42482823139524])@(2,), ZZGate@(1, 3)]
	[None, U1qGate([3.141592658324123, 4.302517087376481])@(1,), None, U1qGate([0.7853981417237897, 2.213435475525513])@(3,)]
	[None, ZZGate@(1, 3), None, ZZGate@(1, 3)]
	[None, U1qGate([4.712388959617491, 6.248839684147036])@(1,), None, U1qGate([5.202667749675468, 3.7842317795539957])@(3,)]
	[ZZGate@(0, 3), None, None, ZZGate@(0, 3)]
	[None, None, None, U1qGate([0.39269911899764576, 2.2134353628165813])@(3,)]
	[ZZGate@(0, 3), None, None, ZZGate@(0, 3)]
	[ZZGate@(0, 2), None, ZZGate@(0, 2), RZGate([0.9

In [7]:
# Save circuit to qasm file
out_circuit.save('QFT_0101.qasm')

## References<a id="references"></a>

1. M. Nielsen and I. Chuang, Quantum Computation and Quantum Information, Cambridge Series on Information and the Natural Sciences (Cambridge University Press, Cambridge, 2000).